In [2]:
#!/usr/bin/env python
import os
import logging
import time
from math import isinf
from statistics import mean
import argparse
import torch
import visdom
import numpy as np
import brambox.boxes as bbb
import lightnet as ln
from lightnet.engine import Engine
from dataset import *



In [45]:
#!/usr/bin/env python
#
#   Copyright EAVISE
#   Example: Perform a single image detection with the Lightnet tiny yolo network
#
"""
Detection Module!!

"""
import os
import argparse
import logging
import cv2
import torch
from torchvision import transforms as tf
import brambox.boxes as bbb
import lightnet as ln


import javabridge 
import bioformats
javabridge.start_vm(class_path=bioformats.JARS, max_heap_size='8G')




log = logging.getLogger('lightnet.detect')

# Parameters
CLASSES = 20
#NETWORK_SIZE = (416, 416)
NETWORK_SIZE = (1482, 2535)
LABELS = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
          'bus', 'car', 'cat', 'chair', 'cow',
          'diningtable', 'dog', 'horse', 'motorbike', 'person',
          'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
CONF_THRESH = .20
NMS_THRESH = .4





# Functions

def post_transform(boxes, scale, pad):
    for box in boxes:
        box.x_top_left -= pad[0]
        box.y_top_left -= pad[1]

        box.x_top_left *= scale
        box.y_top_left *= scale
        box.width *= scale
        box.height *= scale
    return boxes



def create_network(weights):
    """ Create the lightnet network """
    net = ln.models.Yolo(CLASSES, 
                         conf_thresh = CONF_THRESH,
                         nms_thresh = NMS_THRESH,)
    
    net.postprocess.append(ln.data.transform.TensorToBrambox(NETWORK_SIZE, LABELS))
    net.load(weights)
    
    net = net.to(device)
    return net


def detect(net, img_path):
    """ Perform a detection """
    # Load image
    print ("Got to here", img_path)
    img = cv2.imread(img_path)
    im_h, im_w = img.shape[:2]
    
    print (im_h, im_w)

    img_tf = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tf = ln.data.transform.Letterbox.apply(img_tf, dimension=NETWORK_SIZE)
    print (type(img_tf))
    img_tf = tf.ToTensor()(img_tf)
    img_tf.unsqueeze_(0)
    img_tf = img_tf.to(device)
    

    
    """
    filepath = "C://Users//William//Desktop//Box Testing Baseline//Baseline_1//Baseline_1_MMStack_1-Pos_000_002.ome.tif"
    
    image_open = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=3, rescale=False))
    image_para = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=4, rescale=False))
    image_perp = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=5, rescale=False))
    
    img = np.dstack([image_open, image_para, image_perp])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/np.amax(img)
    img = img*255
    img = img.astype('uint8')
    
    im_h, im_w = img.shape[:2]
    img_tf = ln.data.transform.Letterbox.apply(img, dimension = NETWORK_SIZE)
    img_tf = tf.ToTensor()(img_tf)
    img_tf.unsqueeze_(0)
    img_tf = img_tf.to(device)
    """
    
    
    
    print (img.shape, type(img_tf))

    # Run detector
    with torch.no_grad():
        out = net(img_tf)
        
    #print (out)
    #out = ln.data.transform.ReverseLetterbox.apply(out, NETWORK_SIZE, (im_w, im_h)) # Resize bb to true image dimensions
    #tsfm = ln.data.transform.ReverseLetterbox(network_size = NETWORK_SIZE, image_size = (im_w, im_h))
    
    #print (tsfm.apply({"data": out, "network_size" : NETWORK_SIZE, "image_size" : (im_w, im_h)}, network_size = NETWORK_SIZE, image_size = (im_w, im_h)))
    
    net_w, net_h = NETWORK_SIZE

    if im_w == net_w and im_h == net_h:
        scale = 1
    elif im_w / net_w >= im_h / net_h:
        scale = im_w/net_w
    else:
        scale = im_h/net_h
        
    pad = int((net_w - im_w/scale) / 2), int((net_h - im_h/scale) / 2)

    converted_boxes = []
    for b in out:
        converted_boxes.append(post_transform(b, scale, pad))
    
    out = converted_boxes
    #print (converted_boxes)

    return img, out



def output_ROIs (detections):
    for ROIs in detections:
        print (f'X: {ROIs.x_top_left}, Y: {ROIs.y_top_left}, Width: {ROIs.width}, Height: {ROIs.height} {ROIs.class_label}, {ROIs.object_id}')


# Main
if __name__ == '__main__':
    """parser = argparse.ArgumentParser(description='Run an image through the lightnet yolo network')
    parser.add_argument('weight', help='Path to weight file')
    parser.add_argument('image', help='Path to image file(s)', nargs='*')
    parser.add_argument('-c', '--cuda', action='store_true', help='Use cuda')
    parser.add_argument('-s', '--save', action='store_true', help='Save image in stead of displaying it')
    parser.add_argument('-l', '--label', action='store_true', help='Print labels and scores on the image')
    args = parser.parse_args()
    """
    
    #Dog
    image_path = [f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}basic{os.sep}dog.jpg"]
    #weights_path = f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}weights{os.sep}darknet19_448.conv.23.pt"
    
    #image_path = [f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Box Testing Baseline{os.sep}JPEGImages{os.sep}Baseline_1{os.sep}Baseline_1_MMStack_1-Pos_000_00003.jpg"]
    weights_path = f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}yolo-voc{os.sep}backup{os.sep}final.pt"
    
    save_check = False
    show_label = True
    
    
    use_cuda = True
    
    
    # Parse Arguments
    device = torch.device('cpu')
    if use_cuda:
        if torch.cuda.is_available():
            log.debug('CUDA enabled')
            device = torch.device('cuda')
        else:
            log.error('CUDA not available')

    # Network
    network = create_network(weights_path)
    #print(network)
    #print()
    network.training = False
    print("Training", network.training)
    
    
    # Detection
    if len(image_path) > 0:
        print("Taking this path!")
        for img_name in image_path:
            log.info(img_name)
            image, output = detect(network, img_name)
            
            print (output[0])
            print (output[0][0].width)
            output_ROIs(output[0])

            image = bbb.draw_boxes(image, output[0], show_labels=show_label)
            
            #image = cv2.rectangle(image, (0,200),(10,500),(255,0,0), 20)
            if save_check:
                cv2.imwrite('detections.png', image)
            else:
                cv2.imshow('image', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
    else:
        log.error('Invalid file path')
        """
        while True:
            try:
                img_path = input('Enter image path: ')    
            except (KeyboardInterrupt, EOFError):
                print('')
                break
        
            if not os.path.isfile(img_path):
                log.error(f'\'{img_path}\' is not a valid path')
                break

            image, output = detect(network, img_path)
            image = bbb.draw_boxes(image, output[0], show_labels=show_labels)
            if save_check:
                cv2.imwrite('detections.png', image)
            else:
                cv2.imshow('image', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            """


INFO       Loaded weights from C:\Users\William\Desktop\Lightnet\examples\yolo-voc\backup\final.pt


Training False
Taking this path!


INFO       C:\Users\William\Desktop\Lightnet\examples\basic\dog.jpg


Got to here C:\Users\William\Desktop\Lightnet\examples\basic\dog.jpg
576 768
<class 'numpy.ndarray'>
(576, 768, 3) <class 'torch.Tensor'>
[Detection {class_label = car, object_id = None, x = 475.36076670040484, y = 83.28681680161944, w = 199.27805541497975, h = 83.75262525303644, confidence = 0.8246340751647949}, Detection {class_label = dog, object_id = None, x = 147.62779921558703, y = 256.20344129554655, w = 158.6321799089069, h = 197.98650999493927, confidence = 0.6342375874519348}, Detection {class_label = motorbike, object_id = None, x = 61.96438512145749, y = 82.2682818825911, w = 48.649050322621456, h = 44.451824234564775, confidence = 0.6162376403808594}, Detection {class_label = aeroplane, object_id = None, x = 485.48589321862346, y = 372.2284286437247, w = 278.7211222165992, h = 195.79989562246965, confidence = 0.49685728549957275}, Detection {class_label = car, object_id = None, x = 518.3626012145749, y = 194.49772267206478, w = 217.60414030870444, h = 195.60978618421052, c

In [3]:
log = logging.getLogger('lightnet.VOC.train')


class TrainEngine(Engine):
    def start(self):
        self.params.to(self.device)
        if self.valid_loader is not None:
            self.epoch_end()(self.test)

        self.train_loss = {'tot': [], 'coord': [], 'conf': [], 'cls': []}
        self.plot_train_loss = ln.engine.LinePlotter(self.visdom, 'train_loss', opts=dict(xlabel='Batch', ylabel='Loss', title='Training Loss', showlegend=True, legend=['Total loss', 'Coordinate loss', 'Confidence loss', 'Class loss']))
        self.plot_valid_loss = ln.engine.LinePlotter(self.visdom, 'valid_loss', name='Total loss', opts=dict(xlabel='Batch', ylabel='Loss', title='Validation Loss', showlegend=True))
        self.plot_lr = ln.engine.LinePlotter(self.visdom, 'learning_rate', name='Learning Rate', opts=dict(xlabel='Batch', ylabel='Learning Rate', title='Learning Rate Schedule'))
        self.plot_valid_pr = ln.engine.LinePlotter(self.visdom, 'valid_pr', name='latest', opts=dict(xlabel='Recall', ylabel='Precision', title='Validation PR', xtickmin=0, xtickmax=1, ytickmin=0, ytickmax=1, showlegend=True))
        self.best_map = 0

        self.dataloader.change_input_dim()
        self.optimizer.zero_grad()

    def process_batch(self, data):
        data, target = data
        data = data.to(self.device)

        loss = self.network(data, target)
        loss.backward()

        self.train_loss['tot'].append(self.network.loss.loss_tot.item())
        self.train_loss['coord'].append(self.network.loss.loss_coord.item())
        self.train_loss['conf'].append(self.network.loss.loss_conf.item())
        self.train_loss['cls'].append(self.network.loss.loss_cls.item())

    def train_batch(self):
        self.optimizer.step()
        self.optimizer.zero_grad()

        tot = mean(self.train_loss['tot'])
        coord = mean(self.train_loss['coord'])
        conf = mean(self.train_loss['conf'])
        cls = mean(self.train_loss['cls'])
        self.train_loss = {'tot': [], 'coord': [], 'conf': [], 'cls': []}

        self.plot_train_loss(np.array([[tot, coord, conf, cls]]), np.array([self.batch]))
        self.log(f'{self.batch} Loss:{tot:.5f} (Coord:{coord:.2f} Conf:{conf:.2f} Cls:{cls:.2f})')

        if isinf(tot):
            log.error('Infinite loss')
            self.sigint = True
            return

        self.scheduler.step(self.batch, epoch=self.batch)
        self.plot_lr(np.array([self.optimizer.param_groups[0]['lr'] * self.batch_size]), np.array([self.batch]))

    @Engine.batch_end(5000)
    def backup(self):
        self.params.save(os.path.join(self.backup_folder, f'weights_{self.batch}.state.pt'))
        log.info(f'Saved backup')

    @Engine.batch_end(10)
    def resize(self):
        self.dataloader.change_input_dim()

    def test(self):
        log.info('Start testing')
        self.network.eval()
        tot_loss = 0
        anno, det = {}, {}

        with torch.no_grad():
            for idx, (data, target) in enumerate(self.valid_loader):
                data = data.to(self.device)
                output, loss = self.network(data, target)
                tot_loss += loss.item()*len(target)

                key_val = len(anno)
                anno.update({key_val+k: v for k,v in enumerate(target)})
                det.update({key_val+k: v for k,v in enumerate(output)})

                if self.sigint:
                    self.network.train()
                    return

        pr = bbb.pr(det, anno)
        m_ap = round(bbb.ap(*pr)*100, 2)
        loss = tot_loss/len(anno)
        self.log(f'Loss:{loss:.5f} mAP:{m_ap}%')
        self.plot_valid_loss(np.array([loss]), np.array([self.batch]))
        self.plot_valid_pr(np.array(pr[0]), np.array(pr[1]), update='replace')

        if m_ap > self.best_map:
            if self.best_map > 0:
                self.plot_valid_pr(None, name=f'best - {self.best_map}%', update='remove', opts=None)
            self.best_map = m_ap
            self.network.save(os.path.join(self.backup_folder, 'best_map.pt'))
            self.plot_valid_pr(np.array(pr[0]), np.array(pr[1]), name=f'best - {self.best_map}%', update='new', opts=dict(legend=[f'best - {self.best_map}%']))

        self.network.train()

    def quit(self):
        if self.batch >= self.max_batches:
            self.params.network.save(os.path.join(self.backup_folder, 'final.pt'))
            return True
        elif self.sigint:
            self.params.save(os.path.join(self.backup_folder, 'backup.state.pt'))
            return True
        else:
            return False




In [4]:
if __name__ == '__main__':
    
    
    """
    parser = argparse.ArgumentParser(description='Train network')
    parser.add_argument('weight', help='Path to weight file', default=None, nargs='?')
    parser.add_argument('-n', '--network', help='network config file')
    parser.add_argument('-b', '--backup', metavar='folder', help='Backup folder', default='./backup')
    parser.add_argument('-c', '--cuda', action='store_true', help='Use cuda')
    parser.add_argument('-v', '--visdom', action='store_true', help='Visualize training data with visdom')
    parser.add_argument('-e', '--visdom_env', help='Visdom environment to plot to', default='main')
    parser.add_argument('-p', '--visdom_port', help='Port of the visdom server', type=int, default=8097)
    args = parser.parse_args()

    """
    
    backup = f"backup{os.sep}"
    weight = f'weights{os.sep}darknet19_448.conv.23.pt'
    #weight = f'backup{os.sep}final.pt'
    network = f'cfg{os.sep}yolo.py'
    use_cuda = True
    use_visdom = False
    visdom_env = 'main'
    visdom_port = 8097
    
    
    
    
    
    # Parse arguments
    device = torch.device('cpu')
    if use_cuda:
        if torch.cuda.is_available():
            log.debug('CUDA enabled')
            device = torch.device('cuda')
        else:
            log.error('CUDA not available')

    if not os.path.isdir(backup):
        if not os.path.exists(backup):
            log.warn('Backup folder does not exist, creating...')
            os.makedirs(backup)
        else:
            raise ValueError('Backup path is not a folder')
    
    if use_visdom:
        visdom = visdom.Visdom(port=visdom_port, env=visdom_env)
    else:
        visdom = None


    params = ln.engine.HyperParameters.from_file(network)
    if weight is not None:
        if weight.endswith('.state.pt'):
            params.load(weight)
        else:
            params.network.load(weight)

    # Dataloaders
    train_loader = ln.data.DataLoader(
        VOCData(params.train_set, params, True),
        batch_size = params.mini_batch_size,
        shuffle = True,
        drop_last = True,
        num_workers = 0,
        pin_memory = True,
        collate_fn = ln.data.list_collate,
    )

    if params.valid_set is not None:
        valid_loader = torch.utils.data.DataLoader(
            VOCData(params.valid_set, params, False),
            batch_size = params.mini_batch_size,
            shuffle = False,
            drop_last = False,
            num_workers = 0,
            pin_memory = True,
            collate_fn = ln.data.list_collate,
        )
    else:
        valid_loader = None

    # Start training
    eng = TrainEngine(
        params, train_loader,
        valid_loader=valid_loader, device=device, visdom=visdom, backup_folder=backup
    )
    b1 = eng.batch
    t1 = time.time()
    eng()
    t2 = time.time()
    b2 = eng.batch
    log.info(f'Training {b2-b1} batches took {t2-t1:.2f} seconds [{(t2-t1)/(b2-b1):.3f} sec/batch]')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
WARNING    Backup folder does not exist, creating...
WARNING    Modules not matching, performing partial update
INFO       Loaded weights from C:\Users\William\Desktop\Lightnet\examples\yolo-voc\weights\darknet19_448.conv.23.pt
INFO       [brambox.boxes.annotations.pickle] No 'keep_ignore' kwarg found, defaulting to False.


FileNotFoundError: [Errno 2] No such file or directory: 'data/train.pkl'

In [ ]:
"""
Detect Module

"""

backup = f"backup{os.sep}"
weight = f'weights{os.sep}darknet19_448.conv.23.pt'
network = f'cfg{os.sep}yolo.py'
use_cuda = True
use_visdom = False
visdom_env = 'main'
visdom_port = 8097


image_path = f'C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}basic{os.sep}dog.jpg' 


# Parse arguments
device = torch.device('cpu')
if use_cuda:
    if torch.cuda.is_available():
        log.debug('CUDA enabled')
        device = torch.device('cuda')
    else:
        log.error('CUDA not available')


params = ln.engine.HyperParameters.from_file(network)
if weight is not None:
    if weight.endswith('.state.pt'):
        params.load(weight)
    else:
        params.network.load(weight)

        
print (params.network)

print (vars(params))


#Hin, Win = 400,400
#target = range(400)
#im_data = torch.autograd.Variable(torch.randn(len(target), 3, Hin, Win))
#output = params.network._forward(im_data)

In [ ]:
#!/usr/bin/env python
import argparse
import logging
import copy
import time
from pathlib import Path
from statistics import mean
import torch
import numpy as np
from tqdm import tqdm
import lightnet as ln
import brambox.boxes as bbb
from dataset import *

log = logging.getLogger('lightnet.VOC.test')


class TestEngine:
    workers = 8
    pin_mem = True
    coco_metric = False

    def __init__(self, params, **kwargs):
        self.params = params
        self.device = kwargs['device']
        self.loss = kwargs['loss']
        self.fast_pr = kwargs['fast_pr']
        self.network = params.network
        self.network.eval()
        self.network.to(self.device)

        self.test_dataloader = torch.utils.data.DataLoader(
            VOCData(params.test_set, params, False),
            batch_size = params.mini_batch_size,
            shuffle = False,
            drop_last = False,
            num_workers = 8,
            pin_memory = True,
            collate_fn = ln.data.list_collate,
        )

    def __call__(self, csv_file):
        if self.loss == 'none':
            anno, det = self.test_none()
        else:
            anno, det = self.test_loss()

        if self.coco_metric:
            m_ap = []
            for i in range(50, 95, 5):
                m_ap.append(self.ap(det, anno, i/100))
            m_ap = round(mean(m_ap), 2)
            if self.csv_file:
                log.error('CSV file is not possible with the coco metric')
        else:
            m_ap = self.ap(det, anno, csv=csv_file)

        print(f'mAP: {m_ap:.2f}%')

    def ap(self, det, anno, iou=.5, csv=None):
        if csv is not None:
            base_path = Path(csv)

        if self.fast_pr:
            pr = bbb.pr(det, anno, iou)

            if csv is not None:
                np.savetxt(str(base_path), np.array(pr), delimiter=',')

            return round(100 * bbb.ap(*pr), 2)
        else:
            aps = []
            for c in tqdm(self.params.class_label_map):
                anno_c = bbb.filter_discard(copy.deepcopy(anno), [ lambda a: a.class_label == c ])
                det_c  = bbb.filter_discard(copy.deepcopy(det), [ lambda d: d.class_label == c ])
                pr = bbb.pr(det_c, anno_c)

                if csv is not None:
                    np.savetxt(str(base_path.with_name(base_path.stem + f'_{c}' + base_path.suffix)), np.array(pr), delimiter=',')

                aps.append(bbb.ap(*pr))

            return round(100 * mean(aps), 2)

    def test_none(self):
        anno, det = {}, {}

        pp = self.network.postprocess
        self.network.postprocess = None

        with torch.no_grad():
            t_net = 0
            t_pp = 0
            for idx, (data, target) in enumerate(self.test_dataloader):
                data = data.to(self.device)
                t1 = time.perf_counter()
                output = self.network(data)
                torch.cuda.synchronize()
                t2 = time.perf_counter()
                output = pp(output)
                torch.cuda.synchronize()
                t3 = time.perf_counter()

                t_net += t2 - t1
                t_pp += t3 - t2

                base_idx = idx*self.params.mini_batch_size
                anno.update({self.test_dataloader.dataset.keys[base_idx+k]: v for k,v in enumerate(target)})
                det.update({self.test_dataloader.dataset.keys[base_idx+k]: v for k,v in enumerate(output)})


            t_net_img = t_net * 1000 / len(self.test_dataloader.dataset)
            t_pp_img = t_pp * 1000 / len(self.test_dataloader.dataset)
            t_tot_img = t_net_img + t_pp_img
            log.info(f'Time:{t_tot_img:.2f}ms/img (Network:{t_net_img:.3f} Post:{t_pp_img:.3f})')

            self.network.postprocess = pp

            return anno, det

    def test_loss(self):
        loss_dict = {'tot': [], 'coord': [], 'conf': [], 'cls': []}
        anno, det = {}, {}

        with torch.no_grad():
            for idx, (data, target) in tqdm(enumerate(self.test_dataloader), total=len(self.test_dataloader)):
                data = data.to(self.device)
                output, loss = self.network(data, target)

                loss_dict['tot'].append(self.network.loss.loss_tot.item()*len(target))
                loss_dict['coord'].append(self.network.loss.loss_coord.item()*len(target))
                loss_dict['conf'].append(self.network.loss.loss_conf.item()*len(target))
                loss_dict['cls'].append(self.network.loss.loss_cls.item()*len(target))
                base_idx = idx*self.params.mini_batch_size
                anno.update({self.test_dataloader.dataset.keys[base_idx+k]: v for k,v in enumerate(target)})
                det.update({self.test_dataloader.dataset.keys[base_idx+k]: v for k,v in enumerate(output)})

        loss_tot = sum(loss_dict['tot'])/len(anno)
        loss_coord = sum(loss_dict['coord'])/len(anno)
        loss_conf = sum(loss_dict['conf'])/len(anno)
        loss_cls = sum(loss_dict['cls'])/len(anno)
        if self.loss == 'percent':
            loss_coord *= 100 / loss_tot
            loss_conf *= 100 / loss_tot
            loss_cls *= 100 / loss_tot
            log.info(f'Loss:{loss_tot:.5f} (Coord:{loss_coord:.2f}% Conf:{loss_conf:.2f}% Class:{loss_cls:.2f}%)')
        else:
            log.info(f'Loss:{loss_tot:.5f} (Coord:{loss_coord:.2f} Conf:{loss_conf:.2f} Class:{loss_cls:.2f})')

        return anno, det


if __name__ == '__main__':
    
    """
    parser = argparse.ArgumentParser(description='Test trained network')
    parser.add_argument('weight', help='Path to weight file', default=None)
    parser.add_argument('--csv', help='Path for the csv file with the results', default=None)
    parser.add_argument('-n', '--network', help='network config file')
    parser.add_argument('-c', '--cuda', action='store_true', help='Use cuda')
    parser.add_argument('-f', '--fast-pr', action='store_true', help='Use faster but less accurate PR computation method')
    parser.add_argument('-l', '--loss', help='How to display loss', choices=['abs', 'percent', 'none'], default='abs')
    parser.add_argument('-t', '--thresh', help='Detection Threshold', type=float, default=None)
    parser.add_argument('-s', '--save', help='File to store network weights', default=None)

    args = parser.parse_args()
    """
    
    
    backup = f"backup{os.sep}"
    weight = f'weights{os.sep}darknet19_448.conv.23.pt'
    network = f'cfg{os.sep}yolo.py'
    use_cuda = True
    use_visdom = False
    visdom_env = 'main'
    visdom_port = 8097
    thresh = None
    save = None
    loss = 'abs'
    csv = None
    fast_pr = False
    
    
    

    device = torch.device('cpu')
    if use_cuda:
        if torch.cuda.is_available():
            log.debug('CUDA enabled')
            device = torch.device('cuda')
        else:
            log.error('CUDA not available')

    params = ln.engine.HyperParameters.from_file(network)
    if weight is not None:
        if weight.endswith('.state.pt'):
            params.load(weight)
        else:
            params.network.load(weight)
    if thresh is not None:
        params.network.postprocess[0].conf_thresh = thresh

    if save is not None:
        params.network.save(save)

    # Start test
    eng = TestEngine(params, device=device, loss=loss, fast_pr=fast_pr)
    eng(csv)


In [ ]:
#!/usr/bin/env python
#
#   Copyright EAVISE
#   Example: Perform a single image detection with the Lightnet tiny yolo network
#

import os
import argparse
import logging
import cv2
import torch
from torchvision import transforms as tf
import brambox.boxes as bbb
import lightnet as ln

log = logging.getLogger('lightnet.detect')

# Parameters
CLASSES = 20
NETWORK_SIZE = [416, 416]
LABELS = ['person']
CONF_THRESH = .25
NMS_THRESH = .4


# Functions
def create_network(weights):
    """ Create the lightnet network """
    net = ln.models.Yolo(CLASSES, 
                         conf_thresh = .001,
                         nms_thresh = .5,)
    net.postprocess.append(ln.data.transform.TensorToBrambox(NETWORK_SIZE, LABELS))
    net = net.to(device)
    return net


def detect(net, img_path):
    """ Perform a detection """
    # Load image
    print ("Got to here", img_path)
    img = cv2.imread(img_path)
    im_h, im_w = img.shape[:2]

    img_tf = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tf = ln.data.transform.Letterbox.apply(img_tf, dimension=NETWORK_SIZE)
    img_tf = tf.ToTensor()(img_tf)
    img_tf.unsqueeze_(0)
    img_tf = img_tf.to(device)
    
    print (img_tf.shape, type(img_tf))

    # Run detector
    with torch.no_grad():
        out = net(img_tf)
    out = ln.data.transform.ReverseLetterbox.apply(out, NETWORK_SIZE, (im_w, im_h)) # Resize bb to true image dimensions

    return img, out


# Main
if __name__ == '__main__':
    """parser = argparse.ArgumentParser(description='Run an image through the lightnet yolo network')
    parser.add_argument('weight', help='Path to weight file')
    parser.add_argument('image', help='Path to image file(s)', nargs='*')
    parser.add_argument('-c', '--cuda', action='store_true', help='Use cuda')
    parser.add_argument('-s', '--save', action='store_true', help='Save image in stead of displaying it')
    parser.add_argument('-l', '--label', action='store_true', help='Print labels and scores on the image')
    args = parser.parse_args()
    """
    
    image_path = [f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}basic{os.sep}dog.jpg"]
    weights_path = f"C:{os.sep}Users{os.sep}William{os.sep}Desktop{os.sep}Lightnet{os.sep}examples{os.sep}basic{os.sep}yolov2-tiny.weights"
    save_check = False
    
    
    use_cuda = True
    
    
    # Parse Arguments
    device = torch.device('cpu')
    if use_cuda:
        if torch.cuda.is_available():
            log.debug('CUDA enabled')
            device = torch.device('cuda')
        else:
            log.error('CUDA not available')

    # Network
    network = create_network(weights_path)
    #print(network)
    #print()
    network.training = False
    print("Training", network.training)
    
    
    # Detection
    if len(image_path) > 0:
        for img_name in image_path:
            log.info(img_name)
            image, output = detect(network, img_name)

            image = bbb.draw_boxes(image, output[0], show_labels=args.label)
            if save_check:
                cv2.imwrite('detections.png', image)
            else:
                cv2.imshow('image', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
    else:
        while True:
            try:
                img_path = input('Enter image path: ')    
            except (KeyboardInterrupt, EOFError):
                print('')
                break
        
            if not os.path.isfile(img_path):
                log.error(f'\'{img_path}\' is not a valid path')
                break

            image, output = detect(network, img_path)
            image = bbb.draw_boxes(image, output[0], show_labels=args.label)
            if args.save:
                cv2.imwrite('detections.png', image)
            else:
                cv2.imshow('image', image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
